In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"


import pandas as pd
import torch
import torch.nn as nn
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from tqdm import tqdm
from accelerate import Accelerator

/home/user10/miniconda3/envs/datamining/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# 读取 CSV 文件
df = pd.read_csv('/home/user10/dataset/olistbr/brazilian-ecommerce/versions/2/olist_order_reviews_dataset.csv',encoding="utf-8", encoding_errors="replace")  # 替换 'your_file.csv' 为你的 CSV 文件路径

df = df.dropna(subset=['review_score'])  # 去掉 'review_score' 为空的行
df = df[df['review_comment_title'].notna() | df['review_comment_message'].notna()]  # 去掉 'review_comment_title' 和 'review_comment_message' 都为空的行

# 输出列名
print(df.columns,len(df))

#3 提取需要的列
#df = df[['review_score', 'review_comment_title', 'review_comment_message']]

99224


In [3]:
model_path = "/home/user10/code/datamining/input/mbart"
tokenizer = MBart50TokenizerFast.from_pretrained(model_path)
model = MBartForConditionalGeneration.from_pretrained(model_path)
# model=nn.DataParallel(model)
acc=Accelerator()
device = acc.device
print(device)
model = model.to(device)
tokenizer.src_lang='pt_XX'
tgt_lang='en_XX'

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


cuda


In [4]:
df['full_review'] = df['review_comment_title'].fillna('') + "." + df['review_comment_message'].fillna('')
texts = df['full_review'].tolist()  # 获取所有要翻译的文本
# input_ids_list=[]
# attention_masks_list=[]
# batch_size=16
print(len(texts),len(df))

42706 42706


In [5]:
input_ids_list=[]
attention_masks_list=[]
# for i in tqdm(range(0, len(texts)), desc="Tokenizing text", unit="batch"):
#     batch_texts = texts[i]
    
#     # 使用 tokenizer 进行批量编码
#     inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True)
    
#     # 将处理后的 input_ids 存储到列表中
#     input_ids_list.append(inputs['input_ids'])
#     attention_masks_list.append(inputs['attention_mask'])
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
# 将所有批次的 input_ids 合并为一个张量
# input_ids = torch.cat(input_ids_list, dim=0).to(device)
# attention_mask = torch.tensor(attention_masks_list, dim=0).to(device)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
def get_sentiment(score):
    if score in [1, 2]:
        return 'negative'
    elif score == 3:
        return 'neutral'
    elif score in [4, 5]:
        return 'positive'

def save_df(texts,df,bacth_no):
    os.system("clear")
    print(len(texts))
    new_df=df[:len(texts)]
    new_df['text']=texts
    new_df['sentiment'] = new_df['review_score'].apply(get_sentiment)
    result_df = new_df[['sentiment', 'text','review_id', 'order_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date','review_answer_timestamp']]
    print(result_df.head(10))
    result_df.to_csv(f'output/translated_reviews_{len(texts)}.csv', index=False)

    print(f"处理完成，新文件已保存为 output/translated_reviews_{len(texts)}.csv")
    print(result_df.columns,len(result_df))

In [9]:
def batch_translate(input_ids,df, attention_mask, batch_size=16):
    translated_texts = []

    bacth_no=0
    # 分批次翻译
    for i in tqdm(range(0, input_ids.size(0), batch_size),desc="batch translation:"):
        batch_input_ids = input_ids[i:i + batch_size]
        batch_attention_mask = attention_mask[i:i + batch_size]

        # 使用模型进行翻译
        translated_tokens = model.generate(batch_input_ids, attention_mask=batch_attention_mask, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])

        # 解码生成的文本
        translated_batch = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        translated_texts.extend(translated_batch)
        bacth_no+=1
        if bacth_no%200==0:
            save_df(translated_texts,df,bacth_no)

    return translated_texts


In [10]:


with torch.no_grad():
#  执行批量翻译
    df['translated_review'] = batch_translate(input_ids,df, attention_mask, batch_size=16)


texts=df["translated_review"]
os.system("clear")



batch translation::   0%|          | 0/2670 [00:00<?, ?it/s]

batch translation::   7%|▋         | 199/2670 [06:23<1:17:02,  1.87s/it]/tmp/ipykernel_13857/2672230255.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['text']=texts
/tmp/ipykernel_13857/2672230255.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['sentiment'] = new_df['review_score'].apply(get_sentiment)
batch translation::   7%|▋         | 200/2670 [06:25<1:17:23,  1.88s/it]

3200
   sentiment                                               text  \
3   positive                  I got it well before my due date.   
4   positive  Thank you lannister stores I loved to buy thro...   
9   positive  I recommend it. I think it's effective. On the...   
12  positive     But a little bit of labor for that good value.   
15  positive  Great recommendation. Reliable customer, good ...   
16  negative  I wish I knew what I had, I'd just gotten it, ...   
19  negative         My product didn't arrive. It was terrible.   
22  positive                                 Great. The top 10.   
24  positive  It's driven by careful attention, unflinching ...   
27  positive  The purchase was easy, the delivery was done w...   

                           review_id                          order_id  \
3   e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4   f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   
9   8670d52e15e00043ae7de4c01cc2fe0

batch translation::  15%|█▍        | 399/2670 [12:41<1:12:51,  1.92s/it]/tmp/ipykernel_13857/2672230255.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['text']=texts
/tmp/ipykernel_13857/2672230255.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['sentiment'] = new_df['review_score'].apply(get_sentiment)
batch translation::  15%|█▍        | 400/2670 [12:42<1:05:00,  1.72s/it]

6400
   sentiment                                               text  \
3   positive                  I got it well before my due date.   
4   positive  Thank you lannister stores I loved to buy thro...   
9   positive  I recommend it. I think it's effective. On the...   
12  positive     But a little bit of labor for that good value.   
15  positive  Great recommendation. Reliable customer, good ...   
16  negative  I wish I knew what I had, I'd just gotten it, ...   
19  negative         My product didn't arrive. It was terrible.   
22  positive                                 Great. The top 10.   
24  positive  It's driven by careful attention, unflinching ...   
27  positive  The purchase was easy, the delivery was done w...   

                           review_id                          order_id  \
3   e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4   f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   
9   8670d52e15e00043ae7de4c01cc2fe0

...

batch translation::  97%|█████████▋| 2599/2670 [1:19:48<02:10,  1.84s/it]

41600
   sentiment                                               text  \
3   positive                  I got it well before my due date.   
4   positive  Thank you lannister stores I loved to buy thro...   
9   positive  I recommend it. I think it's effective. On the...   
12  positive     But a little bit of labor for that good value.   
15  positive  Great recommendation. Reliable customer, good ...   
16  negative  I wish I knew what I had, I'd just gotten it, ...   
19  negative         My product didn't arrive. It was terrible.   
22  positive                                 Great. The top 10.   
24  positive  It's driven by careful attention, unflinching ...   
27  positive  The purchase was easy, the delivery was done w...   

                           review_id                          order_id  \
3   e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4   f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   
9   8670d52e15e00043ae7de4c01cc2fe

/tmp/ipykernel_13857/2672230255.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['text']=texts
/tmp/ipykernel_13857/2672230255.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['sentiment'] = new_df['review_score'].apply(get_sentiment)
batch translation::  97%|█████████▋| 2600/2670 [1:19:51<02:22,  2.04s/it]

处理完成，新文件已保存为 output/translated_reviews_41600.csv
Index(['sentiment', 'text', 'review_id', 'order_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'],
      dtype='object') 41600


batch translation:: 100%|██████████| 2670/2670 [1:22:09<00:00,  1.85s/it]


In [ ]:
print(len(texts))
new_df=df[:len(texts)]
new_df['text']=texts
new_df['sentiment'] = new_df['review_score'].apply(get_sentiment)
result_df = new_df[['sentiment', 'text','review_id', 'order_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date','review_answer_timestamp']]
print(result_df.head(10))
result_df.to_csv(f'output/translated_reviews_{len(texts)}.csv', index=False)

print(f"处理完成，新文件已保存为 output/translated_reviews_{len(texts)}.csv")
print(result_df.columns,len(result_df))

42706
   sentiment                                               text  \
3   positive             .Recebi bem antes do prazo estipulado.   
4   positive  .Parabéns lojas lannister adorei comprar pela ...   
9   positive  recomendo.aparelho eficiente. no site a marca ...   
12  positive   .Mas um pouco ,travando...pelo valor ta Boa.\r\n   
15  positive  Super recomendo.Vendedor confiável, produto ok...   
16  negative  .GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI ...   
19  negative                    Não chegou meu produto .Péssimo   
22  positive                                 Ótimo.Loja nota 10   
24  positive             .obrigado pela atençao amim dispensada   
27  positive  .A compra foi realizada facilmente.\r\nA entre...   

                           review_id                          order_id  \
3   e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4   f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   
9   8670d52e15e00043ae7de4c01cc2fe

/tmp/ipykernel_13857/996640275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['text']=texts
/tmp/ipykernel_13857/996640275.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['sentiment'] = new_df['review_score'].apply(get_sentiment)


处理完成，新文件已保存为 output/translated_reviews_42706.csv
Index(['sentiment', 'text', 'review_id', 'order_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'],
      dtype='object') 42706


: 